In [ ]:
import pandas as pd
from drn_interactions.transforms.spikes import SpikesHandler
from drn_interactions.config import Config
from drn_interactions.stats import auc, p_adjust
from drn_interactions.interactions.popcup import PopulationCoupling
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.loaders import SpontaneousActivityLoader
from drn_interactions.interactions.pairwise import PairwiseCorr, PairwisePartialCorr
from drn_interactions.interactions.runners import InteractionsRunner
from drn_interactions.io import load_derived_generic, load_neurons
from scipy.stats import zscore
from pingouin import pcorr

In [ ]:
sessions = load_neurons()["session_name"].unique()
session = sessions[2]

loader = SpontaneousActivityLoader(session_name=session, shuffle=False)
preprocessor = InteractionsPreprocessor(z=True)
pcorr = PairwisePartialCorr()
corr = PairwiseCorr()
pcup = PopulationCoupling()

runner = InteractionsRunner(loader=loader, preprocessor=preprocessor, pcup=pcup, corr=corr, pcorr=pcorr)


In [ ]:
pcorr = runner.run_pcorr_multi(sessions=sessions)
corr = runner.run_corr_multi(sessions=sessions)
pcup = runner.run_pcup_multi(sessions=sessions)

In [ ]:
pcorr_reps = runner.pcorr_bootstrap_multi(sessions=sessions, n_boot=1000)
corr_reps = runner.corr_bootstrap_multi(sessions=sessions, n_boot=1000)
pcup_reps = runner.pcup_bootstrap_multi(sessions=sessions, n_boot=1000)

In [ ]:
from drn_interactions.config import Config

dd = Config.data_dir / "corrs"

dd.mkdir(exist_ok=True)

pcorr.to_csv(dd / "spont - pcorr.csv", index=False)
corr.to_csv(dd / "spont - corr.csv", index=False)
pcup.to_csv(dd / "spont - pcup.csv", index=False)

pcorr_reps.to_parquet(dd / "spont - pcorr_reps.parquet.gzip", index=False, 
compression="gzip")
corr_reps.to_parquet(dd / "spont - corr_reps.parquet.gzip", index=False,
compression="gzip")
pcup_reps.to_parquet(dd / "spont - pcup_reps.parquet.gzip", index=False,
compression="gzip")